In [1]:
from pathlib import Path

from ase import units, Atoms
from ase.build import molecule
from ase.io import read, write
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
from prefect import flow
from prefect_dask import DaskTaskRunner
from pymatgen.core import Molecule
from pymatgen.io.packmol import PackmolBoxGen

from mlip_arena.models.utils import REGISTRY, MLIPEnum
from mlip_arena.tasks.md import run as MD

## Create initial configuration

In [3]:
h2 = molecule("H2")
o2 = molecule("O2")
h2o = molecule("H2O")

write("h2.xyz", h2)
write("o2.xyz", o2)
write("h2o.xyz", h2o)

In [4]:
h2 = Molecule.from_file("h2.xyz")
o2 = Molecule.from_file("o2.xyz")
h2o = Molecule.from_file("h2o.xyz")

In [5]:
molecules = []

for m, number in zip([h2, o2], [128, 64]):
    molecules.append(
        {
            "name": m.composition.to_pretty_string(),
            "number": number,
            "coords": m,
        }
    )

In [6]:
tolerance = 2.0
input_gen = PackmolBoxGen(
    tolerance=tolerance,
    seed=1,
)
margin = 0.5 * tolerance

a = 30

packmol_set = input_gen.get_input_set(
    molecules=molecules,
    box=[margin, margin, margin, a - margin, a - margin, a - margin],
)
packmol_set.write_input(".")
packmol_set.run(".")

atoms = read("packmol_out.xyz")
atoms.cell = [a, a, a]
atoms.pbc = True

print(atoms)

write(f'{atoms.get_chemical_formula()}.extxyz', atoms)

Atoms(symbols='H256O128', pbc=True, cell=[30.0, 30.0, 30.0])


## Run workflow

In [2]:
atoms = read("H256O128.extxyz")
print(atoms)

Atoms(symbols='H256O128', pbc=True, cell=[30.0, 30.0, 30.0])


In [3]:
nodes_per_alloc = 1
gpus_per_alloc = 4
ntasks = 1

# cluster_kwargs = dict(
#     cores=1,
#     memory="64 GB",
#     shebang="#!/bin/bash",
#     account="matgen",
#     walltime="02:00:00",
#     job_mem="0",
#     job_script_prologue=[
#         "source ~/.bashrc",
#         "module load python",
#         "source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena",
#     ],
#     job_directives_skip=["-n", "--cpus-per-task", "-J"],
#     job_extra_directives=[
#         "-J combustion-water",
#         "-q regular",
#         f"-N {nodes_per_alloc}",
#         "-C gpu",
#         f"-G {gpus_per_alloc}",
#         f"--exclusive",
#         # "--time-min=00:30:00",
#         # "--comment=1-00:00:00",
#         # "--signal=B:USR1@60",
#         # "--requeue",
#         # "--open-mode=append"
#     ],
#     death_timeout=86400
# )

# cluster = SLURMCluster(**cluster_kwargs)

cluster_kwargs = {
    "cores": 1,
    "memory": "64 GB",
    "shebang": "#!/bin/bash",
    "account": "matgen",
    "walltime": "00:30:00",
    "job_mem": "0",
    "job_script_prologue": [
        "source ~/.bashrc",
        "module load python",
        "source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena",
    ],
    "job_directives_skip": ["-n", "--cpus-per-task", "-J"],
    "job_extra_directives": [f"-N {nodes_per_alloc}", "-q debug", "-C gpu", "-J combustion-water"],
}
cluster = SLURMCluster(**cluster_kwargs)

print(cluster.job_script())
cluster.adapt(minimum_jobs=2, maximum_jobs=2)
client = Client(cluster)

#!/bin/bash

#SBATCH -A matgen
#SBATCH --mem=0
#SBATCH -t 00:30:00
#SBATCH -N 1
#SBATCH -q debug
#SBATCH -C gpu
#SBATCH -J combustion-water
source ~/.bashrc
module load python
source activate /pscratch/sd/c/cyrusyc/.conda/mlip-arena
/pscratch/sd/c/cyrusyc/.conda/mlip-arena/bin/python -m distributed.cli.dask_worker tcp://128.55.64.19:39737 --name dummy-name --nthreads 1 --memory-limit 59.60GiB --nanny --death-timeout 60



/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38693 instead
  warnings.warn(
2024-10-08 03:30:05,696 - distributed.scheduler - ERROR - Task run-b8e0881eb1e8959f855ada9515884ada marked as failed because 4 workers died while trying to run it
2024-10-08 03:30:05,723 - distributed.scheduler - ERROR - Task run-3307095fc41590db704148cc499bad14 marked as failed because 4 workers died while trying to run it


In [4]:
@flow(task_runner=DaskTaskRunner(address=client.scheduler.address), log_prints=True)
def combustion(atoms: Atoms):
    futures = []

    for model in MLIPEnum:
        future = MD.submit(
            atoms=atoms,
            calculator_name=model,
            calculator_kwargs=None,
            ensemble="nvt",
            dynamics="nose-hoover",
            time_step=None,
            ase_md_kwargs=dict(ttime=25 * units.fs, pfactor=None),
            total_time=1000_000,
            temperature=[300, 3000, 3000, 300],
            pressure=None,
            md_velocity_seed=0,
            traj_file=Path(REGISTRY[model.name]["family"])
            / f"{model.name}_{atoms.get_chemical_formula()}.traj",
            traj_interval=1000,
            restart=True,
        )

        futures.append(future)
    
    return [future.result() for future in futures]

In [5]:
results = combustion(atoms)

01:25:41.097 | INFO    | prefect.engine - Created flow run 'green-turaco' for flow 'combustion'

01:25:41.101 | INFO    | prefect.engine - View at https://app.prefect.cloud/account/f7d40474-9362-4bfa-8950-ee6a43ec00f3/workspace/d4bb0913-5f5e-49f7-bfc5-06509088baeb/runs/flow-run/784dc0e0-9f4f-4320-8bfa-4c5e8d3b35fb

01:25:41.409 | INFO    | prefect.task_runner.dask - Connecting to existing Dask cluster SLURMCluster(91d5c261, 'tcp://128.55.64.19:39737', workers=0, threads=0, memory=0 B)

03:30:05.739 | ERROR   | Flow run 'green-turaco' - Encountered exception during execution: KilledWorker('run-3307095fc41590db704148cc499bad14', <WorkerState 'tcp://128.55.65.2:37007', name: SLURMCluster-1, status: closed, memory: 0, processing: 0>, 3)
Traceback (most recent call last):
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/flow_engine.py", line 652, in run_context
    yield self
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/flow_engine.py", line 696, in run_flow_sync
    engine.call_flow_fn()
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/flow_engine.py", line 675, in call_flow_fn
    result = call_with_parameters(self.flow.fn, self.parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect/utilities/callables.py", line 206, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_892374/2043615938.py", line 26, in combustion
    return [future.result() for future in futures]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_892374/2043615938.py", line 26, in <listcomp>
    return [future.result() for future in futures]
            ^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/prefect_dask/task_runners.py", line 132, in result
    future_result = self._wrapped_future.result(timeout=timeout)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/distributed/client.py", line 328, in result
    return self.client.sync(self._result, callback_timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/distributed/client.py", line 336, in _result
    raise exc.with_traceback(tb)
distributed.scheduler.KilledWorker: Attempted to run task 'run-3307095fc41590db704148cc499bad14' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://128.55.65.2:37007. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

03:30:06.504 | ERROR   | Flow run 'green-turaco' - Finished in state Failed("Flow run encountered an exception: KilledWorker: Attempted to run task 'run-3307095fc41590db704148cc499bad14' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://128.55.65.2:37007. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.")

KilledWorker: Attempted to run task 'run-3307095fc41590db704148cc499bad14' on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://128.55.65.2:37007. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.